In [2]:
from pathlib import Path
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from backend.main import generate_video, generate_script, generate_paper, generate_assets #, publish_post_command
from dataclasses import dataclass

# For notebook usage we need to apply the nest_asyncio patch
# !pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
root_path = Path("./pipeline")
root_path

PosixPath('pipeline')

In [5]:
paper_id = "2411.09003"

In [ ]:
paper_root = root_path / paper_id
paper_root.mkdir(parents=True,exist_ok=True)

FileNotFoundError: [Errno 2] No such file or directory: 'pipeline/2411.09003'

In [ ]:
paper_md_target = (paper_root / "paper").with_suffix(".md")
paper_md = generate_paper(paper_id)
paper_md_target.write_text(paper_md)

In [ ]:
script_txt_target = (paper_root / "script").with_suffix(".txt")
script_txt = generate_script(paper_md)

In [ ]:
total_duration = generate_assets(
    script=script_txt,
    use_path=False,
    mp3_output=(paper_root / "audio").with_suffix(".wav").absolute().as_posix(),
    srt_output=(paper_root / "subtitles").with_suffix(".srt").absolute().as_posix(),
    rich_output=(paper_root / "rich").with_suffix(".json").absolute().as_posix(),
)

In [ ]:
output_video = (paper_root / "video").with_suffix(".mp4")
generate_video(
    input_dir = paper_root,
    output_video = output_video,
)

In [ ]:
output_video = (paper_root / "video").with_suffix(".mp4")
output_video